<a href="https://colab.research.google.com/github/LoloziAnas/deeplearning/blob/master/PredictingThePriceOfUsedCars_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
# Import the dataset.
dataset = pd.read_csv('toyota.csv')

In [ ]:
# Cleaning the data.

print(dataset['transmission'].unique())
#['Manual' 'Automatic' 'Semi-Auto' 'Other']

#Exclude the case that contains Other.
dataset = dataset[dataset.transmission != 'Other']

print(dataset['fuelType'].unique())
#['Petrol' 'Other' 'Hybrid' 'Diesel']

#Exclude the case that contains Other.
dataset = dataset[dataset.fuelType != 'Other']

# Replace the qualitative with quantitative variables.

#====================Transmission========================
dataset['transmission'].replace('Manual',0,inplace=True)
dataset['transmission'].replace('Automatic',1,inplace=True)
dataset['transmission'].replace('Semi-Auto',2,inplace=True)

#====================Fuel type============================
dataset['fuelType'].replace('Petrol',0,inplace=True)
dataset['fuelType'].replace('Diesel',1,inplace=True)
dataset['fuelType'].replace('Hybrid',2,inplace=True)

# Changing the type of the features fuelType and transmission from Series to integer.
dataset['fuelType']=dataset['fuelType'].astype(int)
dataset['transmission']=dataset['transmission'].astype(int)

# Splitting data to features and target.
feature_cols=['year','transmission','mileage','tax','mpg','engineSize']
x = dataset[feature_cols]
y = dataset.price

In [ ]:
# Splitting data into training and test data.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [ ]:

from sklearn import svm
from sklearn.model_selection import GridSearchCV

clf = svm.SVR()
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf']}  

regr = GridSearchCV(clf,param_gridو refit = True, verbose = 3)

# fitting the model for grid search 
regr.fit(x_train,y_train)


In [ ]:

# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 
